- 有点乱，大家根据自己习惯瞎改改吧
- kernel替换完成后，不可逆，需要刷新重启
- 测试结果都是取得稳定状态下的迭代速度，大概在20步之后

In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import torch
import sys
import importlib
# 如果测huggingface中没有模型，需要把路径导进来，不然import了kernel，使用AutoModel加载模型也没有用处
sys.path.append('/mnt/workspace/mdy/models')
module = importlib.import_module('glm-4-9b-chat.modeling_chatglm')
from transformers import AutoConfig, AutoModelForCausalLM
import time
from copy import deepcopy
from tqdm import tqdm

/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [2]:

# config = AutoConfig.from_pretrained('/mnt/workspace/mdy/models/Qwen2.5-7B-Instruct')
dtype = torch.bfloat16
bs = 8
seq_len = 2048
iters = 50
model_path = '/mnt/workspace/mdy/models/glm-4-9b-chat'
model_path = '/mnt/workspace/mdy/models/Qwen2.5-0.5B'
# model_path = '/mnt/workspace/mdy/models/Llama-3.2-1B-Instruct'
# model_path = '/mnt/workspace/mdy/models/Meta-Llama-3.1-8B-Instruct'

model = AutoModelForCausalLM.from_pretrained(model_path,
                                            _attn_implementation='flash_attention_2',
                                           device_map='cuda', torch_dtype=dtype, trust_remote_code=True)
# model = module.ChatGLMForConditionalGeneration.from_pretrained(model_path, _attn_implementation='flash_attention_2',
#                                            device_map='cuda', torch_dtype=dtype, trust_remote_code=True)
input_ids = torch.randint(10000, 20000, (bs, seq_len)).cuda()
pos_ids = torch.arange(seq_len)[None, :].expand(bs, -1).contiguous().cuda()
labels = input_ids
grad_list = []
for p in model.parameters():
    grad_list.append(p)
def zero_gard():
    for p in grad_list:
        if p.grad is not None:
            p.grad.zero_()


In [3]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    out = model(input_ids, position_ids=pos_ids, labels=labels)
    out.loss.backward()
t1 = time.time() - t
t1

100%|██████████| 50/50 [00:25<00:00,  1.92it/s]


25.984844207763672

In [4]:
print(grad_list[10].grad[:5])
zero_gard()

tensor([[ 0.0243,  0.0352, -0.0635,  ..., -0.0128,  0.0159, -0.0400],
        [-0.0015, -0.0074,  0.0083,  ...,  0.0076,  0.0071,  0.0061],
        [ 0.0243,  0.0206, -0.0703,  ..., -0.0151,  0.0304, -0.0129],
        [ 0.0088, -0.0027, -0.0204,  ...,  0.0100,  0.0102, -0.0249],
        [-0.0082, -0.0056, -0.0114,  ..., -0.0065,  0.0028, -0.0093]],
       device='cuda:0', dtype=torch.bfloat16)


In [5]:
from mdy_triton.replace_kernel import *

In [6]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    out = model(input_ids, position_ids=pos_ids,labels=labels)
    out.loss.backward()
t2 = time.time() - t
t2

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:16<00:00,  3.07it/s]


16.31452441215515

In [7]:
print(grad_list[10].grad[:5])
zero_gard()

tensor([[ 0.0222,  0.0312, -0.0381,  ..., -0.0076,  0.0101, -0.0306],
        [ 0.0061,  0.0005, -0.0025,  ...,  0.0121,  0.0172, -0.0030],
        [ 0.0038,  0.0204, -0.0703,  ..., -0.0114,  0.0200, -0.0184],
        [ 0.0050, -0.0091,  0.0045,  ...,  0.0010, -0.0020, -0.0177],
        [-0.0042,  0.0030,  0.0131,  ..., -0.0114, -0.0003, -0.0129]],
       device='cuda:0', dtype=torch.bfloat16)


In [ ]:
torch.cuda.empty_cache()
bs = 16
seq_len = 2048
vocab_size = 150000
print('logits显存占用：',(bs * seq_len * vocab_size) / 1e9 * 2,"G")
logits = torch.randn(bs, seq_len, vocab_size, device='cuda', dtype=torch.float16)
logits.requires_grad_(True)
labels = torch.randint(0, vocab_size-1, (bs, seq_len)).cuda()
labels[:, seq_len//2:] = -100
for i in tqdm(range(200)):
    shift_logits = logits[:, :-1].contiguous()
    shift_labels = labels[:, 1:].contiguous()
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(shift_logits.view(-1, vocab_size), shift_labels.view(-1))
    loss.backward()
# 峰值37G， 4倍logits，应该是logits+grad和softmax+grad。然而真正算loss的时候，需要转为float32，占用70G+
# 0.12s/it

logits显存占用： 4.9152 G


100%|██████████| 200/200 [00:22<00:00,  8.87it/s]


In [ ]:
from mdy_triton.core import fast_cross_entropy_loss
torch.cuda.empty_cache()
bs = 16
seq_len = 2048
vocab_size = 150000
print('logits显存占用：',(bs * seq_len * vocab_size) / 1e9 * 2,"G")
extra_ignored_labels = torch.full((seq_len, 1), -100, device='cuda')
logits = torch.randn(bs, seq_len, vocab_size, device='cuda', dtype=torch.float16)
logits.requires_grad_(True)
labels = torch.randint(0, vocab_size-1, (bs, seq_len)).cuda()
labels[:, seq_len//2:] = -100
for i in tqdm(range(200)):
    shift_logits = logits
    shift_labels = torch.hstack((labels[..., 1:], extra_ignored_labels[:labels.shape[0]]))
    loss = fast_cross_entropy_loss(shift_logits, shift_labels)
    loss.backward()
# 重启运行
# 峰值19G， 2倍logits，无softmax中间结果
# 0.03s/it

logits显存占用： 4.9152 G


100%|██████████| 200/200 [00:03<00:00, 53.94it/s]
